In [1]:
%pip install -q --proxy http://proxy:80 timm

Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
sys.path.append("../")
from search_space.RegNetTIMM import RegNetCfg, RegNet
import random
import numpy as np
import time
import torch
from IPython.display import clear_output
random_seed = 42
random.seed(random_seed)
# Set seed for NumPy
np.random.seed(random_seed)
# Set seed for PyTorch
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
# Additional steps if using CuDNN (optional, for GPU acceleration)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append("../")
from evaluation import main
from data_processor import DataProcessor
#from search_space.RegNet import RegNet
import torch
from trainer import Trainer, TrainerDistillation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from IPython.display import clear_output
#from search_space.config import cfg, load_cfg

#Dataset must be in the folder datasets
(train_x, train_y), (valid_x, valid_y), (test_x), metadata = main.load_datasets("AddNIST", truncate=False)

data_processor = DataProcessor(train_x, train_y, valid_x, valid_y, test_x, metadata)
train_loader, valid_loader, test_loader = data_processor.process()

In [3]:
device

device(type='cuda')

In [4]:
#regnety_004=RegNetCfg(w0=48, wa=27.89, wm=2.09, group_size=8, depth=16, se_ratio=0.25,downsample=None)

regnety_004=RegNetCfg(w0=24, wa=48, wm=2.4, group_size=8, depth=12, se_ratio=0.25,downsample="avg",preact=False)

In [3]:
chromosomes={'olive_wasp': {'ws': [8, 72, 144, 288, 592, 1216],
  'bs': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
  'gs': [8, 8, 8, 8, 8, 8],
  'ds': [1, 1, 3, 4, 9, 2],
  'num_stages': 6,
  'total_size_mb': None,
  'h': 1,
  'w': 1,
  'flops': 2891040,
  'params': 16601152,
  'acts': 11584,
  'WA': 48.0,
  'W0': 8,
  'WM': 2.05,
  'DEPTH': 20},
 'hospitable_pronghorn': {'ws': [40, 80, 176, 368],
  'bs': [1.0, 1.0, 1.0, 1.0],
  'gs': [8, 8, 8, 8],
  'ds': [1, 3, 5, 8],
  'num_stages': 4,
  'total_size_mb': None,
  'h': 1,
  'w': 1,
  'flops': 625946,
  'params': 3458714,
  'acts': 5146,
  'WA': 24.0,
  'W0': 40,
  'WM': 2.0999999999999996,
  'DEPTH': 17},
 'spectral_hare': {'ws': [56, 128, 280, 640],
  'bs': [1.0, 1.0, 1.0, 1.0],
  'gs': [8, 8, 8, 8],
  'ds': [1, 3, 6, 4],
  'num_stages': 4,
  'total_size_mb': None,
  'h': 1,
  'w': 1,
  'flops': 1008582,
  'params': 5639414,
  'acts': 5798,
  'WA': 40.0,
  'W0': 56,
  'WM': 2.249999999999999,
  'DEPTH': 14},
 'loyal_tiger': {'ws': [24, 64, 168],
  'bs': [1.0, 1.0, 1.0],
  'gs': [8, 8, 8],
  'ds': [1, 3, 5],
  'num_stages': 3,
  'total_size_mb': None,
  'h': 1,
  'w': 1,
  'flops': 85258,
  'params': 470250,
  'acts': 1386,
  'WA': 24.0,
  'W0': 24,
  'WM': 2.6499999999999977,
  'DEPTH': 9},
 'astute_badger': {'ws': [24, 56, 144, 352],
  'bs': [1.0, 1.0, 1.0, 1.0],
  'gs': [8, 8, 8, 8],
  'ds': [1, 4, 8, 3],
  'num_stages': 4,
  'total_size_mb': None,
  'h': 1,
  'w': 1,
  'flops': 269906,
  'params': 1538370,
  'acts': 3090,
  'WA': 16.0,
  'W0': 24,
  'WM': 2.4499999999999984,
  'DEPTH': 16},
 'quizzical_caiman': {'ws': [56, 152, 392, 1040],
  'bs': [1.0, 1.0, 1.0, 1.0],
  'gs': [8, 8, 8, 8],
  'ds': [1, 3, 9, 2],
  'num_stages': 4,
  'total_size_mb': None,
  'h': 1,
  'w': 1,
  'flops': 1528842,
  'params': 8941290,
  'acts': 7498,
  'WA': 48.0,
  'W0': 56,
  'WM': 2.6499999999999977,
  'DEPTH': 15},
 'zircon_toad': {'ws': [16, 40, 96, 232, 576],
  'bs': [1.0, 1.0, 1.0, 1.0, 1.0],
  'gs': [8, 8, 8, 8, 8],
  'ds': [1, 1, 2, 5, 3],
  'num_stages': 5,
  'total_size_mb': None,
  'h': 1,
  'w': 1,
  'flops': 585964,
  'params': 3328012,
  'acts': 3884,
  'WA': 40.0,
  'W0': 16,
  'WM': 2.4499999999999984,
  'DEPTH': 12},
 'hypersonic_sambar': {'ws': [40, 96, 248],
  'bs': [1.0, 1.0, 1.0],
  'gs': [8, 8, 8],
  'ds': [1, 4, 8],
  'num_stages': 3,
  'total_size_mb': None,
  'h': 1,
  'w': 1,
  'flops': 271408,
  'params': 1507456,
  'acts': 3056,
  'WA': 24.0,
  'W0': 40,
  'WM': 2.4999999999999982,
  'DEPTH': 13},
 'honest_saluki': {'ws': [16, 32, 64, 128, 256],
  'bs': [1.0, 1.0, 1.0, 1.0, 1.0],
  'gs': [8, 8, 8, 8, 8],
  'ds': [1, 1, 3, 6, 2],
  'num_stages': 5,
  'total_size_mb': None,
  'h': 1,
  'w': 1,
  'flops': 127384,
  'params': 734520,
  'acts': 1944,
  'WA': 16.0,
  'W0': 16,
  'WM': 2.0,
  'DEPTH': 13},
 'neat_puffin': {'ws': [24, 48, 104],
  'bs': [1.0, 1.0, 1.0],
  'gs': [8, 8, 8],
  'ds': [2, 5, 4],
  'num_stages': 3,
  'total_size_mb': None,
  'h': 1,
  'w': 1,
  'flops': 35938,
  'params': 204130,
  'acts': 962,
  'WA': 8.0,
  'W0': 24,
  'WM': 2.0999999999999996,
  'DEPTH': 11}}

In [4]:
len(list(chromosomes.keys())[:])

10

In [6]:
clear_output(wait=True)
for student in list(chromosomes.keys())[2:3]:
    regnet_model=RegNetCfg(w0=chromosomes[student]["W0"], wa=chromosomes[student]["WA"], wm=chromosomes[student]["WM"], group_size=8, depth=chromosomes[student]["DEPTH"], se_ratio=0.25,downsample="avg",preact=False)
    model=RegNet(regnet_model,
                in_chans=3,
                num_classes=20,
                #output_stride=8,
                #global_pool='avg',
                drop_rate=0.1,
                drop_path_rate=0.1,
                zero_init_last=True)
    metadata["experiment_name"]=f"tests_one_model/Generation_1_timm/{student}"
    trainer=TrainerDistillation(model, device, train_loader, valid_loader, metadata)
    model_trained=trainer.train()
    clear_output(wait=True)

[{'out_chs': 56, 'stride': 2, 'dilation': 1, 'depth': 1, 'bottle_ratio': 1.0, 'group_size': 8, 'drop_path_rates': array([0.])}, {'out_chs': 128, 'stride': 2, 'dilation': 1, 'depth': 3, 'bottle_ratio': 1.0, 'group_size': 8, 'drop_path_rates': array([0.00769231, 0.01538462, 0.02307692])}, {'out_chs': 280, 'stride': 2, 'dilation': 1, 'depth': 6, 'bottle_ratio': 1.0, 'group_size': 8, 'drop_path_rates': array([0.03076923, 0.03846154, 0.04615385, 0.05384615, 0.06153846,
       0.06923077])}, {'out_chs': 640, 'stride': 2, 'dilation': 1, 'depth': 4, 'bottle_ratio': 1.0, 'group_size': 8, 'drop_path_rates': array([0.07692308, 0.08461538, 0.09230769, 0.1       ])}]
4


[TRAIN] Epoch:1| Time(data):0.000| Time(train):0.068| Loss:2.7681| Top-1:13.093| Top-5:43.423: 100%|██████████| 703/703 [01:06<00:00, 10.50it/s]
[EVAL] Top-1:13.400| Top-5:42.793: 100%|██████████| 235/235 [00:08<00:00, 28.84it/s]
[TRAIN] Epoch:2| Time(data):0.000| Time(train):0.069| Loss:2.4763| Top-1:18.570| Top-5:56.477: 100%|██████████| 703/703 [01:07<00:00, 10.41it/s]
[EVAL] Top-1:20.480| Top-5:58.027: 100%|██████████| 235/235 [00:08<00:00, 28.65it/s]
[TRAIN] Epoch:3| Time(data):0.000| Time(train):0.069| Loss:2.3479| Top-1:21.548| Top-5:61.731: 100%|██████████| 703/703 [01:07<00:00, 10.38it/s]
[EVAL] Top-1:18.753| Top-5:51.660: 100%|██████████| 235/235 [00:08<00:00, 29.27it/s]
[TRAIN] Epoch:4| Time(data):0.000| Time(train):0.069| Loss:2.2471| Top-1:24.095| Top-5:65.496: 100%|██████████| 703/703 [01:07<00:00, 10.43it/s]
[EVAL] Top-1:25.960| Top-5:67.860: 100%|██████████| 235/235 [00:08<00:00, 28.55it/s]
[TRAIN] Epoch:5| Time(data):0.000| Time(train):0.069| Loss:2.1559| Top-1:27.069|

[EVAL] Best accuracy:58.380001068115234


In [7]:
model

RegNet(
  (stem): ConvNormAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (s1): RegStage(
    (b1): Bottleneck(
      (conv1): ConvNormAct(
        (conv): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNormAct2d(
          8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
      )
      (conv2): ConvNormAct(
        (conv): Conv2d(8, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNormAct2d(
          8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
      )
      (se): SEModule(
        (fc1): Conv2d(8, 8, kernel_size=(1, 1), s

In [9]:
trainer=Trainer(model, device, train_loader, valid_loader, metadata)
model_trained=trainer.train()

0.01
	Epoch   1/2   | Train Acc:  13.44% | Valid Acc:  15.69% | T/Epoch: 45.08s  |
0.005
	Epoch   2/2   | Train Acc:  19.57% | Valid Acc:  20.07% | T/Epoch: 43.97s  |
  Total runtime: 1m,27s


In [6]:
    model=RegNet(regnety_004,
                in_chans=3,
                num_classes=20,
                #output_stride=8,
                #global_pool='avg',
                drop_rate=0.1,
                drop_path_rate=0.1,
                zero_init_last=True)

[{'out_chs': 24, 'stride': 2, 'dilation': 1, 'depth': 1, 'bottle_ratio': 1.0, 'group_size': 8, 'drop_path_rates': array([0.])}, {'out_chs': 56, 'stride': 2, 'dilation': 1, 'depth': 1, 'bottle_ratio': 1.0, 'group_size': 8, 'drop_path_rates': array([0.00909091])}, {'out_chs': 136, 'stride': 2, 'dilation': 1, 'depth': 2, 'bottle_ratio': 1.0, 'group_size': 8, 'drop_path_rates': array([0.01818182, 0.02727273])}, {'out_chs': 328, 'stride': 2, 'dilation': 1, 'depth': 7, 'bottle_ratio': 1.0, 'group_size': 8, 'drop_path_rates': array([0.03636364, 0.04545455, 0.05454545, 0.06363636, 0.07272727,
       0.08181818, 0.09090909])}, {'out_chs': 800, 'stride': 1, 'dilation': 2, 'depth': 1, 'bottle_ratio': 1.0, 'group_size': 8, 'drop_path_rates': array([0.1])}]
5


In [7]:
metadata["experiment_name"]="tests_one_model/deft_bat"
trainer=TrainerDistillation(model, device, train_loader, valid_loader, metadata)
trainer.train()

[TRAIN] Epoch:1| Time(data):0.000| Time(train):0.068| Loss:3.0475| Top-1:6.116| Top-5:27.768:   5%|▍         | 34/703 [00:03<00:56, 11.79it/s]

KeyboardInterrupt: 

[TRAIN] Epoch:1| Time(data):0.000| Time(train):0.068| Loss:3.0475| Top-1:6.116| Top-5:27.768:   5%|▍         | 35/703 [00:19<00:56, 11.79it/s]